In [ ]:
"""
1. Make new dirs: 
rootdir with region name, 
{region name}: fitting files,
others: for other files
log: logs for fitting
download the subdir of region name after that.

2. mosspectra and mosback in sas21
3. pn-spectra and pn_back in sas20

note: don't run mosspectra or pnspectra or backs at same time in same dir!

"""

In [11]:
# for a new region 01r500
import os
from glob import glob

# SRCNAME1 = f''
# SRCNAME2 = f'RGH80'
# workpath = f'/data/yanling/XGAP-ABUN/data/data/{SRCNAME2}/reduction'

SRCNAME1 = f'ID828'
SRCNAME2 = f'SDSSTG828'
workpath = f'/data/yanling/XGAP-ABUN/data/data/{SRCNAME1}/reduction/{SRCNAME2}'

regname = f'R500_01'
savepath = f'{workpath}/{regname}'

ELO=700
EHI=1200


In [12]:
f = open('tmp/esas_exspec.sh', 'w')
f.write(
f'''
#!/bin/bash
heainit
sasinit21
source {workpath}/set_chips_on.txt
source {workpath}/set_sas.txt
#### MOS ####
export M1=mos1S001
export M2=mos2S002

mkdir {regname}
mkdir {regname}/{regname}
mkdir {regname}/img
mkdir {regname}/logs
mkdir {regname}/diagnose

# mosspectra eventfile=${{M1}}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/{regname}-${{M1}}.reg pattern=12 withsrcrem=yes maskdet=${{M1}}-bkgregtdet.fits masksky=${{M1}}-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M1ON}}" -V=7 2>&1 |tee {regname}/logs/mosspectra_${{M1}}.log 
# mosback inspecfile=${{M1}}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M1ON}}" 2>&1  |tee mosback_${{M1}}.log 
mosspectra eventfile=${{M2}}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/{regname}-${{M2}}.reg pattern=12 withsrcrem=yes maskdet=${{M2}}-bkgregtdet.fits masksky=${{M2}}-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M2ON}}" -V=7 2>&1 |tee {regname}/logs/mosspectra_${{M2}}.log
mosback inspecfile=${{M2}}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M2ON}}" 2>&1  |tee mosback_${{M2}}.log 

for name in $M2
do
echo $name

mv $name*-{ELO}-{EHI}* {regname}/img
mv $name-fovt.pi R500_01/R500_01/$name-obj-{SRCNAME2}_R500_01.pi
mv $name-bkg.pi R500_01/R500_01/$name-back-{SRCNAME2}_R500_01.pi
mv $name*.qdp {regname}/diagnose
mv $name*imt* {regname}/img
mv $name*.arf {regname}/{regname}/$name-{SRCNAME2}_{regname}.arf
mv $name*.rmf {regname}/{regname}/$name-{SRCNAME2}_{regname}.rmf
mv $name*imspdet* {regname}/img
done

#### PN ####
heainit
sasinit20
source {workpath}/set_chips_on.txt
source {workpath}/set_sas.txt
export name=pnS003
cp $name-allevc.fits $name-clean.fits
cp $name-allevcoot.fits $name-clean-oot.fits
cp $name-corevc.fits $name-corn.fits
cp $name-corevcoot.fits $name-corn-oot.fits
cp $name-bkgregtdet.fits sas20-pn-test/$name-bkg_region-sky.fits 
cp $name-bkgregtsky.fits sas20-pn-test/$name-bkg_region-det.fits 

for PAT in 0 4
do
pn-spectra prefix='S003' caldb=/data/yanling/sas20/esas-caldb region=spec_reg/R500_01-pnS003.reg mask=1 elow={ELO} ehigh={EHI} pattern=4 quad1=1 quad2=1 quad3=1 quad4=1 2>&1 | tee {regname}/logs/pnspectra_$PAT.log
pn_back prefix=S003 caldb=/data/yanling/sas20/esas-caldb diag=0 elow={ELO} ehigh={EHI} pattern=4 quad1=1 quad2=1 quad3=1 quad4=1 2>&1 | tee {regname}/logs/pnback_$PAT.log

mkdir {regname}/img/pn_$PAT
mkdir {regname}/diagnose/pn_$PAT

mv $name*-{ELO}-{EHI}* {regname}/img/pn_$PAT
mv $name-obj.pi {regname}/{regname}/$name-obj-$PAT-{SRCNAME2}_{regname}.pi
mv $name-obj-oot.pi {regname}/{regname}/$name-obj-oot-$PAT-{SRCNAME2}_{regname}.pi
mv $name-back.pi {regname}/{regname}/$name-back-$PAT-{SRCNAME2}_{regname}.pi
mv $name*.qdp {regname}/diagnose/pn_$PAT
mv $name*imt* {regname}/img/pn_$PAT
mv $name*.arf {regname}/{regname}/$name-$PAT-{SRCNAME2}_{regname}.arf
mv $name*.rmf {regname}/{regname}/$name-$PAT-{SRCNAME2}_{regname}.rmf
mv $name*imspdet* {regname}/img/pn_$PAT

done
'''
)
f.close()

# cd {workpath}
# . codes/mosspectra.sh